# <center> L-TEM analysis of Magnetic Samples with MTIE</center>

This code will align the different images of magnetic samples captured using Lorentz microscopy (underfocus, infocus and overfocus) and calculates the magnetization of the sample using the transport of intensity equation (TIE)

author: Dr. Joseph Vas & Prof. Martial Duchamp

This code is developed based on the MTIE equation for quantifying magnetic domains measured using Lorentz TEM in a JEOL ARM 300 double corrected microscope in a field free conditions by switching off the objective lens. The image focus is done by the loretz lens available with the image corrector. The probe corrector is switched off except for the lens responsible for correcting the 2 and 3 stig.

The main reference for the algorithm is given below. 
<br>[1] V.V Volkov, Y Zhu, Lorentz phase microscopy of magnetic materials, Ultramicroscopy, Volume 98, Issues 2–4, 2004,pp. 271-281, ISSN 0304-3991,https://doi.org/10.1016/j.ultramic.2003.08.026.
<br>[2] https://learnopencv.com/image-alignment-ecc-in-opencv-c-python/

The formulation for the MTIE is given in [1].

The key equations for the calculation are





1. $\phi(r) = F^{-1}[F(k_{z}\delta_{z}I/I)/k_\bot^{2}]$
2. $t{B} = \frac{\hbar}{e}[n_{z}\times \Delta\phi]$

The algorithm used will be as follows,
<br>(1) Import the 3 images - under focus, in focus and over focus images
<br>(2) calculating the homography transform matrix of the underfocus and overfocus images with respect to the infocus image and aligning the out of focus images to the in-focus image
<br>(3) masking the areas with large fresnal frignes to eliminate spurious results
<br>(4) calculating the phase shift to the electron wave due to magnetic fields of the sample using equation 1.
<br>(5) calculating the magnetic field from the phase shift using equation 2.
<br>(6) Overlaying the field vectors on the in-phase images

## <center>This version of the code is optimized for working with Lorentz images with only 1 defocus </center>

<ins>Step 1</ins>. Importing the different dm4 images - underfocus, infocus and overfocus

<ins>importing relevant python libraries</ins>

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import cv2
from scipy  import ndimage
import math
from scipy import ndimage

In [2]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [4]:
import hyperspy.api as hs

<ins>importing the in-focus and out of the focus images collected in L-TEM</ins>

In [5]:
TEM1 = hs.load('out-focus.dm4') #underfocus
TEM2 = hs.load('in-focus.dm4') #in-focus

<ins>Step 2.</ins> Calculating the homography transform matrix of the underfocus and overfocus images with respect to the infocus image and aligning the out of focus images to the in-focus image.
<br> Before the phase information and the fields can be calculated from the L-TEM images, the artifacts brought in due to the defocus needs to be calculated. This is done by calculating the transforamtion- homography which takes into account the translation, rotation and sheer transformations due to the defocus. Another defocus effect is the introduction of fresnal fringes at the edges of the samples. This cannot be easily removed and thus is ignored.

<br>Before the transforms can be done, in-order to reduce the processing time, the images need to be binned down. This is written as a separate function

In [6]:
rebin_order = 8 #the order by which the size is to be binned
bin_sizex = TEM1.data.shape[0]/rebin_order
bin_sizey = TEM1.data.shape[1]/rebin_order

under_focus = TEM1.rebin([bin_sizex,bin_sizey])
in_focus = TEM2.rebin([bin_sizex,bin_sizey])



<ins>aligning the images with respect to the in-focus image</ins>

<br> The transformation matrix is calculated in 2 stages, the first one to find the translation between the images. Once the translation is corrected, the rotation and sheer of the images are corrected for using the homography transform.

This function is used to calculate the translation, rotation and sheer transformation between the in-focus image and out of focus images

In [7]:
def align2d_cv(template_original,image_original,warp_mode): #function for calculating the alignments
    
    template = template_original.copy()
    image = image_original.copy()

    template_mean = np.mean(template)
    image_mean = np.mean(image)

    template[template>template_mean] = template_mean
    image[image>image_mean] = image_mean

    template[template<template_mean] = 0
    image[image<image_mean] = 0

    temp = cv2.normalize(template, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    imag = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    
    #find the shape
    sz = image.shape

    #defining the mask
    mask = temp.copy()
    #mask_mean = np.mean(mask)
    #mask[mask>0.9*mask_mean] = 1
    #mask[mask<0.9*mask_mean] = 0

    mask[:,:] = 1
    mask[0,0] = 0
    
    
    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        warp_matrix = np.eye(3, 3, dtype=np.float32)
    else :
        warp_matrix = np.eye(2, 3, dtype=np.float32)
          
     # Specify the number of iterations.
    number_of_iterations = 3000;
    
    # Specify the threshold of the increment
    termination_eps = 1e-10;
    
    # Define termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)

    # Run the ECC algorithm. The results are stored in warp_matrix.
    (cc, warp_matrix) = cv2.findTransformECC(temp,imag,warp_matrix, warp_mode, criteria,mask,gaussFiltSize = 5)
    
    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        # Use warpPerspective for Homography 
        im2_aligned = cv2.warpPerspective (image_original, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    else :
        # Use warpAffine for Translation, Euclidean and Affine
        im2_aligned = cv2.warpAffine(image_original, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP);

    return(im2_aligned)      

In [8]:
under_focus_translated = align2d_cv(in_focus.data, under_focus.data, cv2.MOTION_TRANSLATION)
under_focus_affined = align2d_cv(in_focus.data, under_focus_translated, cv2.MOTION_HOMOGRAPHY)



In [9]:
vmin_mul = 0.
vmax_mul = 2.5

<ins>Plotting to check the alignments</ins>

In [10]:
fig, ax = plt.subplots(nrows = 1, ncols= 3)
image =under_focus_affined
ax[0].imshow(image, cmap='gray', \
                 vmin=vmin_mul*np.mean(image), vmax=vmax_mul*np.mean(image))
ax[0].set_title('under_focus')

image =in_focus.data
ax[1].imshow(image, cmap='gray', \
                 vmin=vmin_mul*np.mean(image), vmax=vmax_mul*np.mean(image))
ax[1].set_title('in_focus')
ax[2].set_title('difference')

image =in_focus.data-under_focus_affined
ax[2].imshow(image, cmap='gray', \
                 vmin=vmin_mul*np.mean(image), vmax=vmax_mul*np.mean(image))

<ins>Step 3.</ins> Creating the mask containing the region of interest
<br> Three things need to be done for a good mask. <br>(1) Remove the high frequency noise within the region of interest. This is done by fft filtering here. <br>(2) Remove the outer edges which are not of interest to the study. This is done by using a rectangle to cut off that region. <br>(3) remove a buffer layer from the edge to remove the areas influenced by fresnal fringes

In [11]:
mask = in_focus.deepcopy()
mask_mean = np.mean(mask.data)

fig, ax = subplots(nrows = 2, ncols = 2)
ax[0,0].imshow(mask.data,cmap = 'gray')

mask.data[mask.data<0.98*mask_mean] = 1  
mask.data[mask.data>0.98*mask_mean] = 0

ax[0,1].imshow(mask.data,cmap = 'gray')

#building the fft filter to eliminate the high frequency noise
Y = (np.fft.fftshift(np.fft.fft2(mask.data)))
circle = np.zeros([Y.shape[0],Y.shape[1]])

for i in range(0,circle.shape[0]):
    for j in range(0,circle.shape[1]):
        if np.sqrt((float(i)-(circle.shape[0]/2))**2+(float(j)-(circle.shape[1]/2))**2)< circle.shape[0]/3:
            circle[i,j] = 1
            
filtered_Y = (circle*Y)
filtered_mask = abs(np.fft.ifft2(np.fft.ifftshift(filtered_Y)))
mask.data = filtered_mask

mask_mean = np.mean(mask.data)
mask.data[mask.data<0.9*mask_mean] = 0
mask.data[mask.data>0.9*mask_mean] = 1

ax[1,0].imshow(mask.data,cmap = 'gray')

#removing the box from the edge

theta = 11 #in degrees
c1y =-25 # left vertical line
c2y = 440 # right vertical line
c3x = 100 # left horizondal line
c4x = 405 # left horizondal line

for i in range(0,circle.shape[0]):
    for j in range(0,circle.shape[1]):
        y = j-(tan(theta*pi/180)*i+c1y)
        if y<0:
            #print(y)
            mask.data[i,j] = 0
        y = j-(tan(theta*pi/180)*i+c2y)
        if y>0:
            #print(y)
            mask.data[i,j] = 0  
        y = i-((1/tan(-(90-theta)*pi/180))*j+c3x)
        if y<0:
            #print(y)
            mask.data[i,j] = 0
        y = i-((1/tan(-(90-theta)*pi/180))*j+c4x)
        if y>0:
            #print(y)
            mask.data[i,j] = 0
#ax[2].imshow(mask.data,cmap = 'gray')    

#removing the fringes at edges

diff = abs(np.gradient(mask.data,axis =0))
#ax[1,0].imshow(diff,cmap = 'gray')

diff_2=ndimage.uniform_filter(diff, size=10, mode='constant')
diff_2_mean = np.mean(diff_2)
diff_2[diff_2>0.6*diff_2_mean] = 1


mask.data = mask.data - diff_2
mask.data[mask.data<0] = 0
ax[1,1].imshow(mask.data,cmap = 'gray')


In [13]:
pixel_dimensions = 18.747e-9*in_focus.original_metadata.ImageList.TagGroup0.ImageData.Calibrations.Dimension.TagGroup0.Scale*rebin_order

In [14]:
pixel_dimensions

1.8939607664465904e-08

<ins>Step 4.</ins> Calculating the phase of the wave due to magnetic sample. 
<br>Since we need to find the log of the intensity difference, having zero intensity in the data creates singularities. This has to be corrected before calculating the log.

<br>The calculating involves constants which can depend on the accelarating voltage which needs to be corrected for relativistic effects. One of the important constants is the defocus used for the measurements. This needs to be inputted.

In [15]:
#preventing singularity while taking log
under_focus_affined = under_focus_affined +0.001
in_focus_data = in_focus.data + 0.001

average_intensity = (under_focus_affined+in_focus_data)/2

#need to change with actual change of focus
df = 3000e-9*5e3 

#constants to be inputted
h = 6.626e-34
m = 9.1e-31
electron_charge = 1.6e-19
V = 388.06e3 # accelarating voltage corrected for relativistic effects
lamda = h/math.sqrt(2*m*electron_charge*V)
kz = 2*pi/lamda #this is for vacuum since the wave transformation between under focus and overfocus is in vacuum
t = 5e-9, #thickness of the sample

#calculating the part of equation 1 which will be FFTed
dz_logI = (under_focus_affined-in_focus_data)/(average_intensity*df)
#dz_logI = (log_over-log_under)/(df)
#dz_logI[mask.data<0.2] = 0 #applying mask
#dz_logI[dz_logI==0] = 1/kz

fft_dI = np.fft.fft2(kz*dz_logI)
fft_dI_shift = np.fft.fftshift(fft_dI)

#finding the k space vectors
pixel_dimensions = 18.747e-9*in_focus.original_metadata.ImageList.TagGroup0.ImageData.Calibrations.Dimension.TagGroup0.Scale*rebin_order
field_of_view = dz_logI.shape[0]*pixel_dimensions
#pixel_dimensions = 1

FreqCompRows = np.fft.fftfreq(dz_logI.shape[0],d=pixel_dimensions)
FreqCompCols = np.fft.fftfreq(dz_logI.shape[1],d=pixel_dimensions)


k_perp1 = zeros(dz_logI.shape)

for i in range(0,k_perp1.shape[0]):
    for j in range(0,k_perp1.shape[1]):
        k_perp1[i,j]= np.sqrt(FreqCompRows[i]*FreqCompRows[i]+FreqCompCols[j]*FreqCompCols[j])
        #if k_perp[i,j]< 1e6:
            #k_perp[i,j] = inf

#k_perp1[0,0] = 0.5e5 #removing the singularity caused when k_perp becomes 0
k_perp = np.fft.fftshift(k_perp1)+np.max(k_perp1)/0.5e2 # use a sufficiently small number to avoid the low frequency noise 
#k_perp[k_perp<5e5] = 5e5
fft_part = (fft_dI_shift/(k_perp**2))

#calculalating the phase of the wave when passing throught the sample
phi_crop = abs(np.fft.ifft2(np.fft.ifftshift(fft_part)))


In [32]:
np.log10(k_perp1)

/var/folders/fz/hslxnj6x7pg9scd9025ggjq00000gn/T/ipykernel_2851/3781148640.py:1: RuntimeWarning: divide by zero encountered in log10
  np.log10(k_perp1)


array([[      -inf, 5.01335906, 5.31438906, ..., 5.49048032, 5.31438906,
        5.01335906],
       [5.01335906, 5.16387406, 5.36284406, ..., 5.51335906, 5.36284406,
        5.16387406],
       [5.31438906, 5.36284406, 5.46490405, ..., 5.57033074, 5.46490405,
        5.36284406],
       ...,
       [5.49048032, 5.51335906, 5.57033074, ..., 5.64099531, 5.57033074,
        5.51335906],
       [5.31438906, 5.36284406, 5.46490405, ..., 5.57033074, 5.46490405,
        5.36284406],
       [5.01335906, 5.16387406, 5.36284406, ..., 5.51335906, 5.36284406,
        5.16387406]])

Plotting phase

In [33]:
vmin_mul = 0.
vmax_mul = 2.5

fig, ax = plt.subplots(nrows = 1, ncols= 2)
image1 =abs(phi_crop)
#image2 = k_perp
image2 = abs(fft_part)
ax[0].set_title('Phase of the electron')
ax[0].imshow(log10(image1), cmap='gray', \
                 vmin=vmin_mul*np.mean(image1), vmax=vmax_mul*np.mean(image1))
ax[1].imshow(image2, cmap='gray', \
                 vmin=vmin_mul*np.mean(image2), vmax=vmax_mul*np.mean(image2))

<ins>Step 5.</ins> Calculating the magnetic information within the sample using equation 2
<br>The code gives the cumulative magnetic moment inside the sample

Calculates the X gradient and Y gradient of the image

In [18]:
def gradients(image,pixel_size):
    dim = image.shape
    grad_X = np.zeros((dim[0],dim[1]))
    grad_Y = np.zeros((dim[0],dim[1]))
    for i in range (0,dim[0]-1):
        for j in range (0,dim[1]-1):
            grad_X[i,j] =(image[i,j+1]-image[i,j])/pixel_size
            grad_Y[i,j] =-(image[i+1,j]-image[i,j])/pixel_size
    
    return(grad_X,grad_Y)

In [19]:
coeff = h/(2*pi*electron_charge)
(grad_X,grad_Y) = gradients(phi_crop,pixel_dimensions)

Bx = coeff*grad_Y/t
By = -coeff*grad_X/t


<ins>Step 6.</ins> Plotting the magnetic map within the specimen

In [20]:
angle = 84*np.pi/180
Bx_real = Bx*np.cos(angle)-By*cos(np.pi/2 -angle)
By_real = Bx*np.sin(angle)+By*sin(np.pi/2 -angle)
Bxreal_masked = np.ma.masked_where(mask.data < 1,Bx_real)
Byreal_masked = np.ma.masked_where(mask.data < 1,By_real)

In [23]:
fig, ax = plt.subplots(nrows=1, ncols = 2)
ax[0].imshow(Bxreal_masked, cmap ='gray')
ax[1].imshow(Byreal_masked, cmap ='gray')

In [21]:
Bx_sum = sum(Bxreal_masked[mask == True].data)/Bxreal_masked[mask == True].data.shape
By_sum = sum(Byreal_masked[mask == True].data)/Byreal_masked[mask == True].data.shape
print('Bx = ', Bx_sum[0])
print('By = ', By_sum[0])

Bx =  0.21248667933551146
By =  -0.04872853541768957


In [22]:
Bxreal_masked[mask == True].data.shape

(23952,)

In [25]:
Bx_masked = np.ma.masked_where(mask.data < 1,Bx)
By_masked = np.ma.masked_where(mask.data < 1,By)

x_elements = Bx_masked.shape[0]
y_elements = By_masked.shape[0]
X,Y = np.meshgrid(np.arange(0,x_elements),np.arange(0,y_elements))


In [26]:
image = under_focus_affined
#image = in_focus.data
fig, ax = plt.subplots(nrows = 1, ncols= 1)
ax.set_title('Phase of the electron')
ax.imshow(image, cmap='gray', \
                 vmin=vmin_mul*np.mean(image), vmax=vmax_mul*np.mean(image))
Q=plt.quiver(X,Y,Bx_masked,By_masked, scale =75, color='g')
ax.set_xlim([300,400])
ax.set_ylim([330,180])

(330.0, 180.0)

In [27]:
def RGB_phase(phase,mag,mag_max):
    color = np.zeros([phase.shape[0],phase.shape[1],4])
    phase = phase+180 # to change the range from [-180, 180] to [0, 360]
    for i in range(0, phase.shape[0]):
        for j in range(0,phase.shape[1]):
            #print(phase)
            if 0 <= phase[i,j] <= 120:
                r = (1/120)*phase[i,j]
                g = (-1/120)*phase[i,j]+1
                b = 0
            elif 120 <= phase[i,j] <= 240:
                r = (-1/120)*(phase[i,j]-120)+1
                g = 0
                b = (1/120)*(phase[i,j]- 120)
            elif 240 <= phase[i,j] <= 360:
                r = 0
                g = (1/120)*(phase[i,j]- 240)
                b = (-1/120)*(phase[i,j]-240)+1
            else:
                r = 0
                g = 0
                b = 0
            if r ==0 and g == 0 and b == 0:
                color[i,j,:] = [r,g,b,0]
            else:
                color[i,j,:] = [r,g,b,3*mag[i,j]/mag_max]
            #color[i,j,:] = [r,g,b,1]
            #color[i,j,:] = [r,g,b,mag[i,j]]
            #color[i,j,:] = [r,g,b,5*mag[i,j]/mag_max]
    return(color)

In [28]:
B = np.sqrt(abs(Bx_masked.data**2+By_masked.data**2))
B_phase = np.arctan2(By, Bx) * 180 / np.pi
Bphase_masked = np.ma.masked_where(mask.data < 1,B_phase)
B_max =np.max(B[Bphase_masked.mask == False])
color =RGB_phase(Bphase_masked, B, B_max)


In [29]:
angle = -5
rotated_color = ndimage.rotate(color, angle)
rotated_infocus = ndimage.rotate(in_focus.data,angle)
rotated_Bx = ndimage.rotate(Bx, angle)
rotated_By = ndimage.rotate(By, angle)
rotated_diff = ndimage.rotate(under_focus_affined-in_focus_data,angle)

fig, ax =plt.subplots(nrows=2, ncols =2)
xlim_min = 25
xlim_max = 500
ylim_min = 375
ylim_max = 170
vmn = -20
vmx = 20

ax[0,0].imshow(rotated_diff,cmap ='gray')
ax[0,0].set_xlim([xlim_min,xlim_max])
ax[0,0].set_ylim([ylim_min,ylim_max])
ax[0,0].set_yticks([])
ax[0,0].set_xticks([])
ax[0,0].set_yticklabels([])
ax[0,0].set_xticklabels([])
ax[0,0].set_title('Difference',size =12)

ax[0,1].imshow(rotated_Bx,cmap ='gray', vmin = vmn, vmax = vmx)
ax[0,1].set_xlim([xlim_min,xlim_max])
ax[0,1].set_ylim([ylim_min,ylim_max])
ax[0,1].set_yticks([])
ax[0,1].set_xticks([])
ax[0,1].set_yticklabels([])
ax[0,1].set_xticklabels([])
ax[0,1].set_title('Bx')

ax[1,0].imshow(rotated_By,cmap ='gray', vmin = vmn, vmax = vmx)
ax[1,0].set_xlim([xlim_min,xlim_max])
ax[1,0].set_ylim([ylim_min,ylim_max])
ax[1,0].set_yticks([])
ax[1,0].set_xticks([])
ax[1,0].set_yticklabels([])
ax[1,0].set_xticklabels([])
ax[1,0].set_title('By')

ax[1,1].imshow(rotated_infocus, cmap ='gray')
ax[1,1].imshow(rotated_color)
ax[1,1].set_xlim([xlim_min,xlim_max])
ax[1,1].set_ylim([ylim_min,ylim_max])
ax[1,1].set_yticks([])
ax[1,1].set_xticks([])
ax[1,1].set_yticklabels([])
ax[1,1].set_xticklabels([])
ax[1,1].set_title('Magnetic field')

Text(0.5, 1.0, 'Magnetic field')

# Unused code

spin_X = B*np.cos(B_phase*np.pi/180)
spin_Y = B*np.sin(B_phase*np.pi/180)
spinX_masked = np.ma.masked_where(mask.data < 1,spin_X)
spinY_masked = np.ma.masked_where(mask.data < 1,spin_Y)
fig, ax =plt.subplots(nrows=1, ncols =1)
ax.imshow(under_focus_affined,cmap='gray',vmin=vmin_mul*np.mean(under_focus_affined), vmax = vmax_mul*np.mean(under_focus_affined))
Q=plt.quiver(X,Y,spinX_masked,spinY_masked, color='g', scale = 500)


The following section contains the functions necessary for running the code above

This function bins down the image to reduce the complexity of the transform. This function though useful is not necessary for the current program since I'm using the rebin function of hyperspy

Calculates the X gradient and Y gradient of the image

# Iterative solution for correction for second part of eqn 12